In [51]:
import pandas as pd
import numpy as np
from pathlib import Path

In [52]:
DATA_DIR = Path("../data/raw")

paths = {
    "customers": DATA_DIR / "customers.csv",
    "sessions": DATA_DIR / "sessions.csv",
    "events": DATA_DIR / "events.csv",
    "orders": DATA_DIR / "orders.csv",
    "order_items": DATA_DIR / "order_items.csv",
    "products": DATA_DIR / "products.csv",
    "reviews": DATA_DIR / "reviews.csv",
}

In [53]:
customers = pd.read_csv(paths["customers"], parse_dates=["signup_date"])
sessions  = pd.read_csv(paths["sessions"],  parse_dates=["start_time"])
events    = pd.read_csv(paths["events"],    parse_dates=["timestamp"])
orders    = pd.read_csv(paths["orders"],    parse_dates=["order_time"])
order_items = pd.read_csv(paths["order_items"])
products    = pd.read_csv(paths["products"])
reviews     = pd.read_csv(paths["reviews"], parse_dates=["review_time"])

In [54]:
events["event_type_norm"] = (
    events["event_type"].astype(str).str.strip().str.lower()
)

In [55]:
evt_session_time = (
    events.groupby("session_id")["timestamp"]
    .agg(first_event_time="min", last_event_time="max")
    .reset_index()
)

sess = sessions.merge(evt_session_time, on="session_id", how="left")

sess["session_end_time"] = sess["last_event_time"].fillna(sess["start_time"])
sess["session_seconds"] = (sess["session_end_time"] - sess["start_time"]).dt.total_seconds()
sess["first_event_delay_seconds"] = (sess["first_event_time"] - sess["start_time"]).dt.total_seconds()

display(sess[["session_seconds", "first_event_delay_seconds"]].describe())

print("Negative session_seconds:", int((sess["session_seconds"] < 0).sum()))
print("Negative first_event_delay_seconds:", int((sess["first_event_delay_seconds"] < 0).sum()))

sess[["session_id","customer_id","start_time","first_event_time","session_end_time",
      "session_seconds","first_event_delay_seconds"]].head()

,session_seconds,first_event_delay_seconds
count,120000.000000,120000.000000
mean,4508.875450,928.945000
std,2526.833834,519.640816
min,60.000000,60.000000
25%,2400.000000,480.000000
50%,4380.000000,900.000000
75%,6420.000000,1380.000000
max,13380.000000,1800.000000


Negative session_seconds: 0
Negative first_event_delay_seconds: 0


,session_id,customer_id,start_time,first_event_time,session_end_time,session_seconds,first_event_delay_seconds
0,1,12360,2021-12-27 00:01:36,2021-12-27 00:08:36,2021-12-27 01:59:36,7080.0,420.0
1,2,13917,2025-01-31 21:29:42,2025-01-31 21:48:42,2025-01-31 23:07:42,5880.0,1140.0
2,3,1022,2024-02-19 00:52:50,2024-02-19 00:57:50,2024-02-19 01:17:50,1500.0,300.0
3,4,2882,2024-08-04 19:54:31,2024-08-04 20:24:31,2024-08-04 20:47:31,3180.0,1800.0
4,5,1286,2022-06-28 13:58:08,2022-06-28 14:19:08,2022-06-28 15:27:08,5340.0,1260.0


In [56]:
sess["session_seconds"] = sess["session_seconds"].clip(lower=0)
sess["first_event_delay_seconds"] = sess["first_event_delay_seconds"].clip(lower=0)

In [ ]:
events["event_type_norm"] = events["event_type"].astype(str).str.strip().str.lower()


evt_counts = (
    events.groupby("session_id")
          .size()
          .rename("event_count")
          .reset_index()
)


evt_by_type = (
    events.pivot_table(
        index="session_id",
        columns="event_type_norm",
        values="event_id",
        aggfunc="count",
        fill_value=0
    )
    .reset_index()
)

event_type_cols = [c for c in evt_by_type.columns if c != "session_id"]

sess = (
    sess.merge(evt_counts, on="session_id", how="left")
        .merge(evt_by_type, on="session_id", how="left")
)

sess["event_count"] = sess["event_count"].fillna(0).astype(int)

for c in event_type_cols:
    sess[c] = sess[c].fillna(0).astype(int)

per_type_sum = sess[event_type_cols].sum(axis=1)
print("Mismatch rows:", int((per_type_sum != sess["event_count"]).sum()))

sess[["session_id", "event_count"] + event_type_cols].head()

Mismatch rows: 0


,session_id,event_count,add_to_cart,checkout,page_view,purchase
0,1,10,3,0,7,0
1,2,8,1,1,5,1
2,3,5,1,1,2,1
3,4,2,0,0,2,0
4,5,6,0,0,6,0


In [64]:
GRACE_MINUTES = 30  # allow a little time after last event for payment processing

sess2 = sess.copy()
sess2["session_end_plus_grace"] = sess2["session_end_time"] + pd.Timedelta(minutes=GRACE_MINUTES)

cand = sess2[["session_id","customer_id","start_time","session_end_plus_grace"]].merge(
    orders[["order_id","customer_id","order_time","total_usd"]],
    on="customer_id",
    how="left"
)

cand = cand[
    (cand["order_time"] >= cand["start_time"]) &
    (cand["order_time"] <= cand["session_end_plus_grace"])
].copy()


sess_purchase = (
    cand.groupby("session_id")
        .agg(
            did_purchase=("order_id", lambda x: int(x.notna().any())),
            amount_usd=("total_usd", "sum"),
            n_orders=("order_id", "nunique")
        )
        .reset_index()
)


sess2 = sess2.merge(sess_purchase, on="session_id", how="left")
sess2["did_purchase"] = sess2["did_purchase"].fillna(0).astype(int)
sess2["amount_usd"] = sess2["amount_usd"].fillna(0.0)
sess2["n_orders"] = sess2["n_orders"].fillna(0).astype(int)

print("Purchase rate:", sess2["did_purchase"].mean())
print("Sessions with >=1 order:", int((sess2["n_orders"] >= 1).sum()))
sess2[["did_purchase","amount_usd","n_orders"]].describe()

Purchase rate: 0.2798833333333333
Sessions with >=1 order: 33586


,did_purchase,amount_usd,n_orders
count,120000.000000,120000.000000,120000.000000
mean,0.279883,37.458877,0.279933
std,0.448944,100.442156,0.449079
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,1.000000,19.540000,1.000000
max,1.000000,2984.580000,2.000000


In [61]:
cand.shape, cand["session_id"].nunique(), orders["order_id"].nunique()

((33592, 7), 33586, 33580)

In [ ]:
base_cols = [
    "session_id", "customer_id",
    "start_time", "session_end_time",
    "session_seconds", "first_event_delay_seconds",
    "event_count",
    "did_purchase", "amount_usd", "n_orders"
]

event_type_cols = [c for c in evt_by_type.columns if c != "session_id"]

context_cols = [c for c in ["device", "source", "country"] if c in sess2.columns]

keep_cols = [c for c in (base_cols + context_cols + event_type_cols) if c in sess2.columns]

session_features = sess2[keep_cols].copy()

session_features["session_seconds"] = pd.to_numeric(session_features["session_seconds"], errors="coerce").fillna(0)
session_features["first_event_delay_seconds"] = pd.to_numeric(session_features["first_event_delay_seconds"], errors="coerce").fillna(0)

session_features["did_purchase"] = session_features["did_purchase"].fillna(0).astype(int)
session_features["amount_usd"] = pd.to_numeric(session_features["amount_usd"], errors="coerce").fillna(0.0)
session_features["n_orders"] = session_features["n_orders"].fillna(0).astype(int)

session_features["event_count"] = session_features["event_count"].fillna(0).astype(int)
for c in event_type_cols:
    if c in session_features.columns:
        session_features[c] = session_features[c].fillna(0).astype(int)

session_features["session_seconds"] = session_features["session_seconds"].clip(lower=0)
session_features["first_event_delay_seconds"] = session_features["first_event_delay_seconds"].clip(lower=0)


print("Rows:", len(session_features), "Cols:", session_features.shape[1])
print("Purchase rate:", session_features["did_purchase"].mean())
session_features[["session_seconds","first_event_delay_seconds","event_count","did_purchase","amount_usd"]].describe()

Rows: 120000 Cols: 17
Purchase rate: 0.2798833333333333


,session_seconds,first_event_delay_seconds,event_count,did_purchase,amount_usd
count,120000.000000,120000.000000,120000.000000,120000.000000,120000.000000
mean,4508.875450,928.945000,6.341317,0.279883,37.458877
std,2526.833834,519.640816,3.424481,0.448944,100.442156
min,60.000000,60.000000,1.000000,0.000000,0.000000
25%,2400.000000,480.000000,3.000000,0.000000,0.000000
50%,4380.000000,900.000000,6.000000,0.000000,0.000000
75%,6420.000000,1380.000000,9.000000,1.000000,19.540000
max,13380.000000,1800.000000,17.000000,1.000000,2984.580000


In [ ]:
OUT_DIR = Path("../data/analysis")
OUT_DIR.mkdir(parents=True, exist_ok=True)

session_features.to_csv(OUT_DIR / "session_features.csv", index=False)
session_features.to_parquet(OUT_DIR / "session_features.parquet", index=False, engine="fastparquet")

OUT_DIR

PosixPath('../data/analysis')